In [2]:
import nltk
import gensim
import numpy as np
import pandas as pd
import re,string,unicodedata

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from gensim.utils import simple_preprocess 
from gensim.models import Phrases, Word2Vec
from sklearn.preprocessing import LabelBinarizer
from nltk.tokenize import word_tokenize,sent_tokenize


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Load the IMDb dataset from the CSV file
imdb_df = pd.read_csv('/imdb_dataset.csv', header=None, names=['review', 'sentiment'])
imdb_df["sentiment"] = np.where(imdb_df["sentiment"] == "positive", 0, 1)

gen_df = pd.read_csv('/gen_dataset.csv', header=None, names=['class', 'text'])
gen_df = gen_df.append(pd.read_csv('/HYP-sarc-notsarc.csv'))
gen_df = gen_df.append(pd.read_csv('/RQ-sarc-notsarc.csv'))

gen_df["class"] = np.where(gen_df["class"] == "notsarc", 0, 1)

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

# Preprocess the dataset
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

<ipython-input-3-4cc59efa8916>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/HYP-sarc-notsarc.csv'))
<ipython-input-3-4cc59efa8916>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/RQ-sarc-notsarc.csv'))


In [4]:
imdb_df['review']=imdb_df['review'].apply(denoise_text)
gen_df['text']=gen_df['text'].apply(denoise_text)

<ipython-input-3-4cc59efa8916>:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [5]:
words, imdb_words, gen_words = [], [], []
for i in imdb_df.review.values:
    words.append(i.split())
    imdb_words.append(i.split())
words=words[1:]

for i in gen_df.text.values:
    words.append(i.split())
    gen_words.append(i.split())

In [6]:
imdb_wv_model = Word2Vec(words, vector_size=500, window=3, min_count=1, workers=16)

In [7]:
print(imdb_wv_model)

Word2Vec<vocab=485482, vector_size=500, alpha=0.025>


In [8]:
print(len(words))

59387


In [25]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(485482)
tokenizer.fit_on_texts(words)
text = tokenizer.texts_to_sequences(words)
text = keras.utils.pad_sequences(text, 75)

print(text[:2])

[[   145    166  42448    703     75   1528   4516   2772   7673    703
    7962    703 154647   1293   2081     17    355     37     99   3229
    1655  11805     43   1401     95    159   1888   1562   1551  15623
      79  11649    196   3075   2146   3154   2091  10361 154648   6038
   15624   2956 105798   8723  15624    341    483     10    168     95
      26  54856    604    823   8723    466   1376  33356    478    366
    1051   2583   1376  54857     57  15623     84    247   3879   3815
  154649     10   1237   4495   4324]
 [   201   2519    252   5587     86    369     26 154652    374   6640
   45740     79 154653   2650   3700   5609    203      9  21813    913
   11806   1916  14464  11483  33357     26    148     57  61470    345
    2524   5587   4964    291      3     18  19547    172    389   2519
       8    138    302     27  11650   1119   3719   2270    121    190
    2027 105800   3857   1095   1080  21183    164   1799  18585    419
      64   3416  27819  39

In [10]:
y = imdb_df["sentiment"]

In [11]:
txt = tokenizer.texts_to_sequences(imdb_words)
txt = keras.utils.pad_sequences(txt, 75)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(np.array(txt), np.array(y), train_size=0.8, stratify=y)

In [13]:
model = keras.models.Sequential([

    # Embedding layer
    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=500,
                            input_length=75,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    
    keras.layers.Dropout(0.2),

    # BiLSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),

    keras.layers.Bidirectional(keras.layers.LSTM(64)),

    keras.layers.Dropout(0.2),

    keras.layers.Dense(200),

    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    
    # Output layer
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 75, 500)           242741000 
                                                                 
 dropout (Dropout)           (None, 75, 500)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 75, 256)          644096    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 200)               2

In [14]:
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 75)

y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
235/235 [==============================] - 52s 205ms/step - loss: 0.4346 - accuracy: 0.8446 - val_loss: 0.4154 - val_accuracy: 0.8472
Epoch 2/10
235/235 [==============================] - 33s 139ms/step - loss: 0.1924 - accuracy: 0.9268 - val_loss: 0.6372 - val_accuracy: 0.7417
Epoch 3/10
235/235 [==============================] - 26s 112ms/step - loss: 0.0583 - accuracy: 0.9839 - val_loss: 0.8033 - val_accuracy: 0.6560
Epoch 4/10
235/235 [==============================] - 23s 97ms/step - loss: 0.0308 - accuracy: 0.9883 - val_loss: 0.8491 - val_accuracy: 0.6693
Epoch 5/10
235/235 [==============================] - 20s 85ms/step - loss: 0.0241 - accuracy: 0.9876 - val_loss: 0.7779 - val_accuracy: 0.6448
Epoch 6/10
235/235 [==============================] - 19s 83ms/step - loss: 0.0160 - accuracy: 0.9901 - val_loss: 0.9308 - val_accuracy: 0.7460
Epoch 7/10
235/235 [==============================] - 20s 85ms/step - loss: 0.0155 - accuracy: 0.9907 - val_loss: 0.9061 - val_accura

In [17]:
gen_wv_model = Word2Vec(gen_words, vector_size=500, window=3, min_count=1, workers=16)

In [18]:
print(gen_wv_model)

Word2Vec<vocab=46908, vector_size=500, alpha=0.025>


In [27]:
# tokenizer = Tokenizer(46908)
# tokenizer.fit_on_texts(gen_words)
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 75)

model = keras.models.Sequential([

    # Embedding layer
    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=500,
                            input_length=75,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    
    keras.layers.Dropout(0.2),

    # BiLSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),

    keras.layers.Bidirectional(keras.layers.LSTM(64)),

    keras.layers.Dropout(0.2),

    keras.layers.Dense(200),

    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    
    # Output layer
    keras.layers.Dense(1, activation='sigmoid')
])


y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y, random_state=42)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
118/118 [==============================] - 39s 260ms/step - loss: 0.4227 - accuracy: 0.8454 - val_loss: 0.4193 - val_accuracy: 0.8472
Epoch 2/10
118/118 [==============================] - 24s 206ms/step - loss: 0.4014 - accuracy: 0.8474 - val_loss: 0.4090 - val_accuracy: 0.8472
Epoch 3/10
118/118 [==============================] - 20s 170ms/step - loss: 0.2303 - accuracy: 0.9094 - val_loss: 0.6440 - val_accuracy: 0.7513
Epoch 4/10
118/118 [==============================] - 17s 144ms/step - loss: 0.0598 - accuracy: 0.9828 - val_loss: 0.7253 - val_accuracy: 0.7684
Epoch 5/10
118/118 [==============================] - 16s 136ms/step - loss: 0.0435 - accuracy: 0.9868 - val_loss: 0.6831 - val_accuracy: 0.7519
Epoch 6/10
118/118 [==============================] - 15s 121ms/step - loss: 0.0291 - accuracy: 0.9881 - val_loss: 0.8002 - val_accuracy: 0.7322
Epoch 7/10
118/118 [==============================] - 13s 112ms/step - loss: 0.0216 - accuracy: 0.9893 - val_loss: 0.7950 - val_ac